In [1]:
%pwd


'c:\\Users\\PC\\Documents\\mlops\\research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PC\\Documents\\mlops'

In [51]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir:Path
    base_model_path:Path
    updated_model_path:Path
    params_image_size:list
    params_learning_rate:float
    params_include_top:bool
    params_weight:str
    params_classes:int

In [52]:
from cnn.constants import *
from cnn.utils.common import read_yaml,make_dir



In [53]:
class ConfigurationManager:
    def __init__(self,config_file=CONFIG_FILE_PATH,params_file=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file)
        self.params=read_yaml(params_file)
        print(f"{self.config }")
        print(f'{self.params}')
        make_dir([self.config.artifacts_root])

    def get_base_model_config(self)->PrepareBaseModelConfig:
        config=self.config.prepare_base_model
        print(f"{config}")


        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weight=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
            
        )

        return prepare_base_model_config
    




       

In [54]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weight,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [55]:

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-12-27 02:55:01,594:INFO:common:YAML file loaded successfully from config\config.yaml]
[2024-12-27 02:55:01,600:INFO:common:YAML file loaded successfully from params.yaml]
{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_urls': 'https://drive.google.com/file/d/1YW96OE4y-Uj1N1t35-NzJkOQXbtzx3hZ/view?usp=sharing', 'local_data': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/base_model', 'base_model_path': 'artifacts/base_model/base_model.h5', 'updated_base_model_path': 'artifacts/base_model/updated_base_model.h5'}}
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 1, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}
[2024-12-27 02:55:01,606:INFO:common:directory created at artifacts]
{'root_dir': 'artifacts/base_model', 'base_model_path': 'artifacts/base_model/base_model.h5', 'updated_base_model_

KeyboardInterrupt: 